# Speed up your Python code
## Python is slow
Compared to low-level languages, Python is typically faster to write, less error prone and easier to debug. However, Python is much harder to optimize — that is, turn into fast machine code — than languages like C or Fortran. 

But still, high productivity languages should be chosen over high speed languages for the great majority of scientific computing tasks

This is because

* Of any given program, relatively few lines are ever going to be time-critical
* For those lines of code that are time-critical, we can achieve C-like speed with minor modifications


Let's start by trying to understand why high level languages like Python are slower than compiled code
## Dynamic typing
Consider this Python operation

In [1]:
a, b = 10, 10
a + b

20

Even for this simple operation, the Python interpreter has a fair bit of work to do

For example, in the statement `a + b`, the interpreter has to know which operation to invoke

If `a` and `b` are strings, then `a + b` requires string concatenation

In [2]:
a, b = 'foo', 'bar'
a + b

'foobar'

If `a` and `b` are lists, then `a + b` requires list concatenation

In [3]:
a, b = ['foo'], ['bar']
a + b

['foo', 'bar']

We say that the operator `+` is *overloaded* — its action depends on the type of the objects on which it acts.

As a result, Python must check the type of the objects and then call the correct operation

This involves substantial overheads

## Static Types

Compiled languages avoid these overheads with explicit, static types

For example, consider the following C code, which sums the integers from 1 to 10

```c
#include <stdio.h>

int main(void) {
    int i;
    int sum = 0;
    for (i = 1; i <= 10; i++) {
        sum = sum + i;
    }
    printf("sum = %d\n", sum);
    return 0;
}
```

The variables `i` and `sum` are explicitly declared to be integers

Hence, the meaning of addition here is completely unambiguous

## Data Access

Another drag on speed for high level languages is data access

To illustrate, let’s consider the problem of summing some data — say, a collection of integers

In the standard Python implementation (CPython), list elements are placed in memory locations that are in a sense contiguous. However, these list elements are more like pointers to data rather than actual data

Hence, there is still overhead involved in accessing the data values themselves. This is a considerable drag on speed

In fact, it’s generally true that memory traffic is a major culprit when it comes to slow execution

Let’s look at some ways around these problems

## Vectorization

When we run batch operations on arrays as batch operators, the result is C or Fortran-like speed, then the code is *vectorized*

**Numpy** is all about vectorization. You'll need to change your way of thinking from procedure to vectors

Many functions provided by NumPy are so-called universal functions — also called [*ufuncs*](https://docs.scipy.org/doc/numpy/reference/ufuncs.html) that operate on ndarrays in an element-by-element fashion. 


In [4]:
import random
import numpy as np
import time

In [5]:
start = time.time()   # Start timing
n = 100_000
sum = 0
for i in range(n):
    x = random.uniform(0, 1)
    sum += x**2
end = time.time() # End timing

print("The squared sum with %d numbers is %f in %f secs"
     %(n, sum, end-start))

The squared sum with 100000 numbers is 33332.973566 in 0.102724 secs


Now compare with the vectorized code


In [6]:
start = time.time()   # Start timing
n = 100_000
x = np.random.uniform(0, 1, n)
np.sum(x**2)
end = time.time() # End timing

print("The squared sum with %d numbers is %f in %f secs"
     %(n, sum, end-start))

The squared sum with 100000 numbers is 33332.973566 in 0.223175 secs


Consider the problem of maximizing a function $f$ of two variables $(x,y)$ over the square $[-a, a]\times[-a, a]$:

$f(x,y)=\frac{cos(x^2+y^2)}{1+x^2+y^2}$ and $a=3$


In [7]:
def f(x, y):
    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)

grid = np.linspace(-3, 3, 1000)
m = -np.inf

start = time.time()
for x in grid:
    for y in grid:
        z = f(x, y)
        if z > m:
            m = z

end = time.time()

print("The maximum value is %f in %f secs"
     %(m, end-start))

The maximum value is 0.999982 in 5.128743 secs


And here’s a vectorized version




In [8]:
def f(x, y):
    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)

grid = np.linspace(-3, 3, 1000)
x, y = np.meshgrid(grid, grid)

start = time.time()
np.max(f(x, y))
end = time.time()

print("The maximum value is %f in %f secs"
     %(m, end-start))

The maximum value is 0.999982 in 0.290105 secs


## Numba

Numba works by generating optimized machine code using the LLVM compiler infrastructure at import time, runtime, or statically (using the included pycc tool). 

With a few annotations, array-oriented and math-heavy Python code can be just-in-time compiled to native machine instructions, similar in performance to C, C++ and Fortran, without having to switch languages or Python interpreters.


In [9]:
from numba import jit

Consider some problems that are difficult to vectorize

We generate the trajectory of a difference equation given an initial condition

Take the difference equation to be the quadratic map: $x_{t+1} = 4x_t(1-x_t)$





In [10]:
def qm(x0, n):
    x = np.empty(n+1)
    x[0] = x0
    for t in range(n):
        x[t+1] = 4 * x[t] * (1 - x[t])
    return x

In [11]:
qm_numba = jit(qm)  # qm_numba is now a 'compiled' version of qm

In [12]:
import time
start = time.time()
qm(0.1, int(10**5))
end = time.time()
time1 = end-start
print("in %f secs"
     %( end-start))

in 0.166576 secs


Let’s time and compare identical function calls with JIT compilation:


In [13]:
start = time.time()
qm_numba(0.1, int(10**5))
end = time.time()
time2 = end-start
print("in %f secs"
     %( end-start))

in 8.842789 secs


The first execution is relatively slow because of JIT compilation (see below)

Next time and all subsequent times it runs much faster:

In [14]:
start = time.time()
qm_numba(0.1, int(10**5))
end = time.time()
time2 = end-start
print("The mean value is %f in %f secs"
     %(np.mean(x), end-start))

The mean value is 0.000000 in 0.000999 secs


In [15]:
time1 / time2  # Calculate speed gain

166.7069911715581

This is a huge progress relative to how simple and clear the implementation is

If you don’t need a separate name for the “numbafied” version of qm, you can just put `@jit` before the function. This is equivalent to `qm = jit(qm)`

In [16]:
@jit 
def qm(x0, n):
    x = np.empty(n+1)
    x[0] = x0
    for t in range(n):
        x[t+1] = 4 * x[t] * (1 - x[t])
    return x

## Numba for vectorization

Numba can also be used to create custom `ufuncs` with the `@vectorize` decorator


In [17]:
from numba import vectorize

@vectorize
def f_vec(x, y):
    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)

In [18]:
np.max(f_vec(x, y))  # Run once to compile

start = time.time()
m = np.max(f_vec(x, y))
end = time.time()
print("The maximum value is %f in %f secs"
     %(m, end-start))

The maximum value is 0.999982 in 0.015956 secs


This is faster than our vectorized version using NumPy’s ufuncs

Why should that be? After all, anything vectorized with NumPy will be running in fast C or Fortran code

The reason is that it’s much less memory intensive

For example, when NumPy computes np.cos($x**2$ + $y**2$) it first creates the intermediate arrays $x**2$ and $y**2$, then it creates the array np.cos($x**2$ + $y**2$)

In our `@vectorize` version using Numba, the entire operator is reduced to a single vectorized process and none of these intermediate arrays are created

We can gain further speed improvements using Numba’s automatic parallelization feature by specifying `target=’parallel’`

In this case, we need to specify the types of our inputs and outputs

In [19]:
@vectorize('float64(float64, float64)', target='parallel')
def f_vec(x, y):
    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)

In [20]:
np.max(f_vec(x, y))  # Run once to compile

start = time.time()
m = np.max(f_vec(x, y))
end = time.time()
print("The maximum value is %f in %f secs"
     %(m, end-start))

The maximum value is 0.999982 in 0.006982 secs


## Cython

Like Numba, Cython provides an approach to generating fast compiled code that can be used from Python. As was the case with Numba, a key problem is the fact that Python is dynamically typed

As you’ll recall, Numba solves this problem (where possible) by inferring type. Cython’s approach is different — programmers add type definitions directly to their “Python” code. As such, the Cython language can be thought of as Python with type definitions

In addition to a language specification, Cython is also a language translator, transforming Cython code into optimized C and C++ code. Cython also takes care of building language extentions — the wrapper code that interfaces between the resulting compiled code and Python

Suppose that we want to compute the sum $\sum_{n}^{i=0} \alpha^i = \frac{1-\alpha^{n+1}}{1-\alpha}$ for given $\alpha$, $n$

Here’s a pure Python function that does the job

In [21]:
def geo_prog(alpha, n):
    current = 1.0
    sum = current
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum

Here’s a C function that will do the same thing

```c
double geo_prog(double alpha, int n) {
    double current = 1.0;
    double sum = current;
    int i;
    for (i = 1; i <= n; i++) {
        current = current * alpha;
        sum = sum + current;
    }
    return sum;
}
```


We’re going to run our Cython code in the Jupyter notebook

In [22]:
%load_ext Cython

In [28]:
%%cython
def geo_prog_cython(double alpha, int n):
    cdef double current = 1.0
    cdef double sum = current
    cdef int i
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum

DistutilsPlatformError: Unable to find vcvarsall.bat

Here cdef is a Cython keyword indicating a variable declaration, and is followed by a type

The `%%cython` line at the top is not actually Cython code — it’s a Jupyter cell magic indicating the start of Cython code

After executing the cell, you can now call the function ```geo_prog_cython``` from within Python

What you are in fact calling is compiled C code with a Python call interface

In [29]:
start = time.time()
v = geo_prog(0.99, int(10**6))
end = time.time()
print("The value is %f in %f secs"
     %(v, end-start))

The value is 100.000000 in 0.229891 secs


In [30]:
start = time.time()
v = geo_prog_cython(0.99, int(10**6))
end = time.time()
print("The value is %f in %f secs"
     %(v, end-start))

NameError: name 'geo_prog_cython' is not defined

### Cython with NumPy arrays

For problem of generating the iterates of the quadratic map, it iterates and returning a time series requires us to work with arrays. 

The natural array type to work with is NumPy arrays. 

In [31]:
%%cython
import numpy as np 

def qm_cython_first_pass(double x0, int n):
    cdef int t
    x = np.zeros(n+1, float)
    x[0] = x0
    for t in range(n):
        x[t+1] = 4.0 * x[t] * (1 - x[t])
    return np.asarray(x)

DistutilsPlatformError: Unable to find vcvarsall.bat

If you run this code and time it, you will see that it’s performance is disappointing — nothing like the speed gain we got from Numba

In [32]:
start1 = time.time()
qm_cython_first_pass(0.1, int(10**5))
end1 = time.time()
print("The mean value is %f in %f secs"
     %(np.mean(x), end1-start1))

start2 = time.time()
qm_numba(0.1, int(10**5))
end2 = time.time()
print("The mean value is %f in %f secs"
     %(np.mean(x), end2-start2))

NameError: name 'qm_cython_first_pass' is not defined

The reason is that working with NumPy arrays incurs substantial Python overheads

We can do better by using Cython’s typed `memoryviews`, which provide more direct access to arrays in memory

When using them, the first step is to create a NumPy array. Next, we declare a memoryview and bind it to the NumPy array



In [33]:
%%cython
import numpy as np
from numpy cimport float_t

def qm_cython(double x0, int n):
    cdef int t
    x_np_array = np.zeros(n+1, dtype=float)
    cdef float_t [:] x = x_np_array
    x[0] = x0
    for t in range(n):
        x[t+1] = 4.0 * x[t] * (1 - x[t])
    return np.asarray(x)

DistutilsPlatformError: Unable to find vcvarsall.bat

In [34]:
qe.util.tic()
qm_cython(0.1, int(10**5))
qe.util.toc()

NameError: name 'qe' is not defined

`cimport` pulls in some compile-time information from NumPy

`cdef float_t [:] x = x_np_array` creates a memoryview on the NumPy array `x_np_array`

the return statement uses `np.asarray(x)` to convert the memoryview back to a NumPy array

In [35]:
start3 = time.time()
qm_cython(0.1, int(10**5))
end3 = time.time()
print("The mean value is %f in %f secs"
     %(np.mean(x), end3-start3))

NameError: name 'qm_cython' is not defined

This is fast, although still slightly slower than `qm_numba`

## Python, Cython? (from Scikit-Learn documentation)

When implementing a new algorithm is thus recommended to start implementing it in Python using Numpy and Scipy by taking care of avoiding looping code using the vectorized idioms of those libraries. In practice this means trying to **replace any nested for loops by calls to equivalent Numpy array methods**.

Sometimes however an algorithm cannot be expressed efficiently in simple vectorized Numpy code. In this case, the recommended strategy is the following:
* **Profile** the Python implementation to find the main bottleneck and isolate it in a dedicated module level function. This function will be reimplemented as a compiled extension module.
* If there exists a well maintained C/C++ implementation of the same algorithm that is not too big, you can write a **Cython wrapper** for it and include a copy of the source code of the library in the source tree
* Otherwise, write an optimized version of your Python function using **Cython or Numba** directly. 
* Once the code is optimized (not simple bottleneck spottable by profiling), check whether it is possible to have **coarse grained parallelism** that is amenable to multi-processing 